In [ ]:
# ##Generate OOS file from verification datasets
# import pandas as pd, geopandas as gpd
# DATA_DIR = 'data'

# df_satmeta = pd.read_csv(f'{DATA_DIR}/raw/verification/verification_pm25_satellite_metadata.csv')
# df_satmeta['split'] = 'oos'

# df_labels = pd.read_csv(f'{DATA_DIR}/raw/verification/verification_submission_format.csv')
# df_grid_meta = gpd.read_file(f'{DATA_DIR}/raw/grid_metadata.csv')[['grid_id','location']]
# df_grid_meta['location']=df_grid_meta.location.map({'Delhi':'dl','Los Angeles (SoCAB)':'la','Taipei':'tpe'})
# df_labels = pd.merge(df_grid_meta,df_labels).sort_values(by=['datetime','grid_id']).reset_index(drop=True)

# df_satmeta.to_csv(f'{DATA_DIR}/raw/verification/verification_interim_satellite_metadata.csv',index=False)
# df_labels.to_csv(f'{DATA_DIR}/raw/verification/verification_interim_submission_format.csv',index=False)

In [3]:
# Generate dummy prod file from the test data
from pathlib import Path

import pandas as pd
import geopandas as gpd

VER_DIR = Path("data/raw/verification")
RAW_DIR = VER_DIR / "raw"

df_satmeta = pd.read_csv(f'{RAW_DIR}/verification_pm25_satellite_metadata.csv', index_col="Unnamed: 0")
df_satmeta['split'] = 'prod'

df_labels = pd.read_csv(f'{RAW_DIR}/verification_pm25_submission_format.csv', index_col="Unnamed: 0")
df_grid_meta = gpd.read_file(f'{RAW_DIR}/grid_metadata.csv')[['grid_id','location']]
df_grid_meta['location']=df_grid_meta.location.map({'Delhi':'dl','Los Angeles (SoCAB)':'la','Taipei':'tpe'})

df_satmeta.to_csv(f'{RAW_DIR}/prod_satellite_metadata.csv', index=False)
df_labels.to_csv(f'{RAW_DIR}/prod_submission_format.csv', index=False)

In [4]:
df_labels

,datetime,grid_id,value
0,2021-12-01T08:00:00Z,3S31A,0.0
1,2021-12-01T08:00:00Z,A2FBI,0.0
2,2021-12-01T08:00:00Z,DHO4M,0.0
3,2021-12-01T08:00:00Z,DJN0F,0.0
4,2021-12-01T08:00:00Z,E5P9N,0.0
...,...,...,...
3490,2022-02-27T08:00:00Z,WT52R,0.0
3491,2022-02-27T08:00:00Z,Z0VWC,0.0
3492,2022-02-27T08:00:00Z,ZP1FZ,0.0
3493,2022-02-27T08:00:00Z,ZZ8JF,0.0


In [5]:
df_satmeta

,granule_id,time_start,time_end,product,location,split,us_url,eu_url,as_url,cksum,granule_size
0,20211201T191000_maiac_la_0.hdf,2021-12-01T17:25:00.000Z,2021-12-01 19:10:00+00:00,maiac,la,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,2549388211,12077808
1,20211202T195000_maiac_la_0.hdf,2021-12-02T18:05:00.000Z,2021-12-02 19:50:00+00:00,maiac,la,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1355983959,12331307
2,20211203T203000_maiac_la_0.hdf,2021-12-03T17:10:00.000Z,2021-12-03 20:30:00+00:00,maiac,la,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1065198164,11852730
3,20211204T194000_maiac_la_0.hdf,2021-12-04T17:55:00.000Z,2021-12-04 19:40:00+00:00,maiac,la,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,361463763,11749840
4,20211205T202000_maiac_la_0.hdf,2021-12-05T17:00:00.000Z,2021-12-05 20:20:00+00:00,maiac,la,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1016093029,11817224
...,...,...,...,...,...,...,...,...,...,...,...
414,20220205T062130_misr_dl_0.nc,2022-02-05T05:24:33.000Z,2022-02-05 06:21:30+00:00,misr,dl,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,203235713,39881610
415,20220207T060854_misr_dl_0.nc,2022-02-07T05:11:56.000Z,2022-02-07 06:08:54+00:00,misr,dl,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,286176554,39278514
416,20220214T061409_misr_dl_0.nc,2022-02-14T05:17:10.000Z,2022-02-14 06:14:09+00:00,misr,dl,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,2630621702,41890403
417,20220221T061919_misr_dl_0.nc,2022-02-21T05:22:21.000Z,2022-02-21 06:19:19+00:00,misr,dl,prod,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,228336879,41771333


# Preprocessing

In [8]:
##Extract NASADEM elevation data (Rerun required only if grid_metadata changes)
!python3 src/data/extract_nasadem.py --data_dir data/raw/verification/

  0%|                                                    | 0/48 [00:00<?, ?it/s]src/data/extract_nasadem.py:56: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  da = (xr.open_rasterio(signed_asset.href))
  2%|▉                                           | 1/48 [00:03<02:34,  3.29s/it]src/data/extract_nasadem.py:56: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  da = (xr.open_rasterio(signed_asset.href))
100%|███████████████████████████████████████████| 48/48 [00:57<00:00,  1.20s/it]
Saved data to data/raw/verification/interim/elevation.csv


In [ ]:
##Extract MAIAC data from aws
!python3 src/data/extract_maiac.py --data_dir data/raw/verification/ --path_satmeta data/raw/verification/raw/prod_satellite_metadata.csv --stage prod --s3url us_url --cleanup --n_jobs 16

In [23]:
##Extract GFS data
!python3 src/data/extract_gfs.py --data_dir data/raw/verification/ --stage prod --labels data/raw/verification/raw/prod_submission_format.csv --n_jobs 4

{
   "status": "ok",
   "request_duration": "0.082954 seconds",
   "code": 200,
   "messages": [],
   "result": {
      "subsetting_available": true,
      "data": [
         {
            "request_type": "T",
            "group_index": 0
         }
      ]
   },
   "request_end": "2022-05-20T13:16:46.131066",
   "request_start": "2022-05-20T13:16:46.048112"
}
Location  dl  data request range:  202111280000/to/202203010000
Found existing file data/raw/verification/interim/tmp/gfs/prod/param_group0/dl//202111280000_202203010000.tar. Skipping request
Found existing file data/raw/verification/interim/tmp/gfs/prod/param_group1/dl//202111280000_202203010000.tar. Skipping request
Found existing file data/raw/verification/interim/tmp/gfs/prod/param_group2/dl//202111280000_202203010000.tar. Skipping request
Found existing file data/raw/verification/interim/tmp/gfs/prod/param_group3/dl//202111280000_202203010000.tar. Skipping request
Found existing file data/raw/verification/interim/tmp/gfs/pro

PROCESSING TASKS | : 100%|████████████████████| 372/372 [00:03<00:00, 93.74it/s]
COLLECTING RESULTS | : 100%|██████████████| 372/372 [00:00<00:00, 268366.20it/s]
QUEUEING TASKS | : 100%|████████████████████| 372/372 [00:00<00:00, 9780.79it/s]
PROCESSING TASKS | : 100%|███████████████████| 372/372 [00:02<00:00, 154.73it/s]
COLLECTING RESULTS | : 100%|██████████████| 372/372 [00:00<00:00, 431971.51it/s]
QUEUEING TASKS | : 100%|████████████████████| 372/372 [00:00<00:00, 7052.28it/s]
PROCESSING TASKS | : 100%|███████████████████| 372/372 [00:02<00:00, 166.19it/s]
COLLECTING RESULTS | : 100%|██████████████| 372/372 [00:00<00:00, 493759.84it/s]
QUEUEING TASKS | : 100%|████████████████████| 372/372 [00:00<00:00, 7870.59it/s]
PROCESSING TASKS | : 100%|███████████████████| 372/372 [00:03<00:00, 110.71it/s]
COLLECTING RESULTS | : 100%|██████████████| 372/372 [00:00<00:00, 473386.25it/s]
QUEUEING TASKS | : 100%|████████████████████| 372/372 [00:00<00:00, 9423.13it/s]
PROCESSING TASKS | : 100%|██

In [24]:
##Create final datasets
!python3 src/data/create_dataset.py --data_dir data/raw/verification/ --stage prod --labels data/raw/verification/raw/prod_submission_format.csv --n_gfs 8 --n_jobs 4
!python3 src/data/create_dataset.py --data_dir data/raw/verification/ --stage prod --labels data/raw/verification/raw/prod_submission_format.csv --n_gfs 10 --n_jobs 4
!python3 src/data/create_dataset.py --data_dir data/raw/verification/ --stage prod --labels data/raw/verification/raw/prod_submission_format.csv --n_gfs 12 --n_jobs 4

QUEUEING TASKS | : 100%|█████████████████| 3495/3495 [00:00<00:00, 18206.86it/s]
PROCESSING TASKS | : 100%|██████████████████| 3495/3495 [00:46<00:00, 75.06it/s]
COLLECTING RESULTS | : 100%|████████████| 3495/3495 [00:00<00:00, 599725.59it/s]
sys:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
Saved data to data/raw/verification/processed/prod/prod_tail8.pkl
QUEUEING TASKS | : 100%|█████████████████| 3495/3495 [00:00<00:00, 13243.50it/s]
PROCESSING TASKS | : 100%|██████████████████| 3495/3495 [00:44<00:00, 79.25it/s]
COLLECTING RESULTS | : 100%|████████████| 3495/3495 [00:00<00:00, 595413.99it/s]
sys:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

# Inference

In [25]:
##Generate predictions 
!python3 src/inference.py --data_dir data/raw/verification/ --model_dir models --stage prod --subformat_path data/raw/verification/raw/prod_submission_format.csv --output_path verification_predictions.csv 

##### RUNNING INFERENCE FOR DATASET 8 ##########
location 1, fold: 0, samples:1492 1.021958744181995,32.542325908801196
location 1, fold: 1, samples:1492 2.605776159248359,22.043073670202315
location 1, fold: 2, samples:1492 2.5705810777384848,27.583542280013877
location 1, fold: 3, samples:1492 2.991694496499436,24.40006052486117
location 1, fold: 4, samples:1492 1.435963642199119,25.028862678147746
location 0, fold: 0, samples:2003 27.70732672930693,388.48937838733946
location 0, fold: 1, samples:2003 38.58883046189182,405.47305485368355
location 0, fold: 2, samples:2003 65.89711139273004,400.7329546267776
location 0, fold: 3, samples:2003 58.36404286243944,228.43931601125553
location 0, fold: 4, samples:2003 38.8291688175228,393.7000871970493
##### RUNNING INFERENCE FOR DATASET 10 ##########
location 1, fold: 0, samples:1492 2.017837937594028,28.010720934088546
location 1, fold: 1, samples:1492 2.917265829914939,20.30920478976976
location 1, fold: 2, samples:1492 2.709267904979812,2

In [27]:
pred = pd.read_csv('verification_predictions.csv')
pred

,datetime,grid_id,value
0,2021-12-01T08:00:00Z,3S31A,19.981446
1,2021-12-01T08:00:00Z,A2FBI,15.372362
2,2021-12-01T08:00:00Z,DHO4M,18.697646
3,2021-12-01T08:00:00Z,DJN0F,13.026814
4,2021-12-01T08:00:00Z,E5P9N,20.588916
...,...,...,...
3490,2022-02-27T08:00:00Z,WT52R,6.509498
3491,2022-02-27T08:00:00Z,Z0VWC,10.994868
3492,2022-02-27T08:00:00Z,ZP1FZ,8.978790
3493,2022-02-27T08:00:00Z,ZZ8JF,4.949437


In [28]:
##compare with generated test submission file
sub=pd.read_csv('data/raw/verification/verification_pm25_labels.csv')
sub

,Unnamed: 0,datetime,grid_id,value
0,0,2021-12-01T08:00:00Z,3S31A,38.752083
1,1,2021-12-01T08:00:00Z,A2FBI,12.240741
2,2,2021-12-01T08:00:00Z,DHO4M,28.818421
3,3,2021-12-01T08:00:00Z,DJN0F,13.259649
4,4,2021-12-01T08:00:00Z,E5P9N,16.920175
...,...,...,...,...
3490,3490,2022-02-27T08:00:00Z,WT52R,6.488095
3491,3491,2022-02-27T08:00:00Z,Z0VWC,14.080851
3492,3492,2022-02-27T08:00:00Z,ZP1FZ,3.276316
3493,3493,2022-02-27T08:00:00Z,ZZ8JF,2.288889


In [38]:
from sklearn.metrics import r2_score

grid = pd.read_csv("data/raw/verification/verification_pm25_grid_metadata.csv", index_col=0)
grid_to_loc = dict(zip(grid.grid_id, grid.location))
sub["location"] = sub.grid_id.apply(lambda x: grid_to_loc[x])
sub

,Unnamed: 0,datetime,grid_id,value,location
0,0,2021-12-01T08:00:00Z,3S31A,38.752083,Los Angeles (SoCAB)
1,1,2021-12-01T08:00:00Z,A2FBI,12.240741,Los Angeles (SoCAB)
2,2,2021-12-01T08:00:00Z,DHO4M,28.818421,Los Angeles (SoCAB)
3,3,2021-12-01T08:00:00Z,DJN0F,13.259649,Los Angeles (SoCAB)
4,4,2021-12-01T08:00:00Z,E5P9N,16.920175,Los Angeles (SoCAB)
...,...,...,...,...,...
3490,3490,2022-02-27T08:00:00Z,WT52R,6.488095,Los Angeles (SoCAB)
3491,3491,2022-02-27T08:00:00Z,Z0VWC,14.080851,Los Angeles (SoCAB)
3492,3492,2022-02-27T08:00:00Z,ZP1FZ,3.276316,Los Angeles (SoCAB)
3493,3493,2022-02-27T08:00:00Z,ZZ8JF,2.288889,Los Angeles (SoCAB)


In [37]:
r2_score(sub.value, pred.value)

0.7444769506846506

In [40]:
filt = sub.location == "Los Angeles (SoCAB)"
r2_score(sub[filt].value, pred[filt].value)

0.4636283119085033

In [42]:
filt = sub.location == "Delhi"
r2_score(sub[filt].value, pred[filt].value)

0.43062096880146117